In [ ]:
import pandas as pd
from mp_api.client import MPRester

API_KEY = "dnq07hvQEKLgvwySSmAi4QGzItrjZKpg"  
mpr = MPRester(API_KEY)
df=pd.read_csv('mp_all_v4.csv')
df=df[df['polymorphs']==1]
mp_ids=[i.split('/')[1].split('.')[0] for i in df['mp_id'].tolist()]
docs=[]
fail=[]
for i in mp_ids:
    try:
       docs.append(mpr.thermo.search(material_ids=[i]))
    except:
        fail.append(i)
        print('failed:',i)

In [ ]:
thermo_docs[0].entries['GGA'].data.get('oxidation_states')

In [ ]:
import pandas as pd
from mp_api.client import MPRester

API_KEY = "dnq07hvQEKLgvwySSmAi4QGzItrjZKpg"
mpr = MPRester(API_KEY)

# Load and filter CSV
df = pd.read_csv('mp_all_v4.csv')
df = df[df['polymorphs'] == 1]
print("Filtered shape:", df.shape)

# Extract mp-IDs cleanly
mp_ids = [i.split('/')[1].split('.')[0] for i in df['mp_id'].tolist()]

# Store oxidation states
results = []
fail = []

for i in mp_ids:
    try:
        doc = mpr.thermo.search(material_ids=[i])
        if not doc:
            fail.append(i)
            print(f"No data for: {i}")
            continue

        entry_types = list(doc[0].entries.keys())
        entry_type = entry_types[0]  # "GGA" or "GGA+U" etc.
        oxi = doc[0].entries[entry_type].data.get("oxidation_states", {})

        results.append({
            "mp_id": i,
            "oxidation_states": oxi,
            "entry_type": entry_type
        })

    except Exception as e:
        print(f"Failed for {i}: {e}")
        fail.append(i)

# Convert to DataFrame
oxi_df = pd.DataFrame(results)
oxi_df.to_csv("mp_oxidation_states.csv", index=False)
print("Saved oxidation states to mp_oxidation_states.csv")


In [ ]:
import pandas as pd
df=pd.read_csv("mp_oxidation_states.csv")
df

In [ ]:
df2=pd.read_csv('mp_all_v4.csv')
df['mp_id']=df['mp_id'].apply(lambda x:'cifs/'+x+'.cif')
dicn={}
for i,j in zip(df['mp_id'].tolist(),df['oxidation_states'].tolist()):
    dicn[i]=j
oxis=[]
for i in df2['mp_id'].tolist():
    try:
        
        oxis.append(dicn[i])
    except:
        oxis.append({})
df2['ox_state']=oxis
df2.to_csv('mp_all_v4.csv',index=False)